In [ ]:
# Installing packages like transformers,datasets and  evaluate
!pip install transformers[sentencepiece] datasets evaluate

In [4]:
# Using transformers pipeline to do simple task like senitment classification

from transformers import pipeline
classifier = pipeline("sentiment-analysis")

classifier(
    [
        "I've been waiting for a HuggingFace course all my life",
        "I love tea",
        "Amazing spiderman is great movie",
        "I hate  eating cold food",
        "Hate is everywhere but I don't hate you"
    ]
)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9705262780189514},
 {'label': 'POSITIVE', 'score': 0.9998308420181274},
 {'label': 'POSITIVE', 'score': 0.9998748302459717},
 {'label': 'NEGATIVE', 'score': 0.9832373261451721},
 {'label': 'POSITIVE', 'score': 0.9962484240531921}]

# Exploring Hugging Face pipelineday165.svg

![]()

In [6]:
# step1 Model input-> Embeddings
## using autoktokenizer from transformers hugging face

from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"   # adding checkpoint to cache it according to model's tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [9]:
raw_inputs = [
    "Learning everyday is my new hobby",
    "I like watching spiderman movies",
    "I hate KKR movies"
]
inputs = tokenizer(raw_inputs, padding=True,return_tensors="pt")               # specifying pytorch tensors to return

In [10]:
print(inputs)

{'input_ids': tensor([[  101,  4083, 10126,  2003,  2026,  2047, 17792,   102],
        [  101,  1045,  2066,  3666,  6804,  2386,  5691,   102],
        [  101,  1045,  5223,  1047, 21638,  5691,   102,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0]])}


In [11]:
# Step2 Embeddings -> Layers
## using some transformer model to convert embeddings into hidden states i.e feautres
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

In [13]:
output = model(**inputs)                           # **input means unpacking the dictionary and givven tensor and attention amsk to the model
print(output.last_hidden_state.shape)

torch.Size([3, 8, 768])


In [14]:
# As we can see hidden state has high dimensional output which is controlled by head


## Note : Head part determine the use case like whether our model does sentinment analysis or text classification or whatever the task we want to do is deteremined by this head layers

In [15]:
# Step3: Hidden state -> Head
## Let perform sentiment classification
from transformers import AutoModelForSequenceClassification
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
classifier = AutoModelForSequenceClassification.from_pretrained(checkpoint)


In [17]:
classifications = classifier(**inputs)

In [19]:
## now convert the logits into probability using softmax layer
import torch
predictions = torch.nn.functional.softmax(classifications.logits, dim=-1)
print(predictions)

tensor([[0.0016, 0.9984],
        [0.0022, 0.9978],
        [0.9982, 0.0018]], grad_fn=<SoftmaxBackward0>)


In [24]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}